In [ ]:
!pip install pandas numpy scipy scikit-learn matplotlib seaborn cvxopt xgboost shap kaggle

!git clone https://github.com/drn-i/WESAD
!git clone https://github.com/lciti/cvxEDA.git
!cp /content/cvxEDA/src/* /content/WESAD/
!rm -rf /content/cvxEDA
!rm -rf /content/sample_data

!mv /content/WESAD/* ./
!rm -rf /content/WESAD

# Line 277
# df['label'] = df[['0', '1', '2']].idxmax(axis=1).astype(int)

# Line 209
# feat_names.append('_'.join([str(row), str(col)]))


Cloning into 'WESAD'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 73 (delta 4), reused 0 (delta 0), pack-reused 61 (from 1)
Receiving objects: 100% (73/73), 1001.25 KiB | 4.02 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Cloning into 'cvxEDA'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 82 (delta 21), reused 35 (delta 11), pack-reused 34 (from 1)
Receiving objects: 100% (82/82), 35.82 KiB | 17.91 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("orvile/wesad-wearable-stress-affect-detection-dataset")

print("Path to dataset files:", path)

!mkdir -p /content/data/WESAD
!cp -r /root/.cache/kagglehub/datasets/orvile/wesad-wearable-stress-affect-detection-dataset/versions/1/WESAD/ /content/data/
!cp -r /kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/ /content/data/

Using Colab cache for faster access to the 'wesad-wearable-stress-affect-detection-dataset' dataset.
Path to dataset files: /kaggle/input/wesad-wearable-stress-affect-detection-dataset
cp: cannot stat '/root/.cache/kagglehub/datasets/orvile/wesad-wearable-stress-affect-detection-dataset/versions/1/WESAD/': No such file or directory


In [ ]:
!python data_wrangling.py

/content/data_wrangling.py:159: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  if norm_type is 'std':
/content/data_wrangling.py:162: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
  elif norm_type is 'minmax':
Processing data for S2...
     pcost       dcost       gap    pres   dres
 0: -1.2090e+04 -1.2038e+04  5e+04  2e+02  2e-01
 1: -1.2090e+04 -2.0215e+04  1e+04  4e+01  5e-02
 2: -1.2098e+04 -1.6237e+04  4e+03  1e+01  2e-02
 3: -1.2098e+04 -1.5682e+04  4e+03  1e+01  1e-02
 4: -1.2102e+04 -1.3289e+04  1e+03  3e+00  4e-03
 5: -1.2099e+04 -1.2800e+04  7e+02  1e+00  2e-03
 6: -1.2095e+04 -1.2543e+04  4e+02  8e-01  9e-04
 7: -1.2089e+04 -1.2333e+04  2e+02  3e-01  4e-04
 8: -1.2102e+04 -1.2158e+04  6e+01  2e-02  3e-05
 9: -1.2126e+04 -1.2148e+04  2e+01  6e-03  7e-06
10: -1.2137e+04 -1.2146e+04  1e+01  2e-03  2e-06
11: -1.2142e+04 -1.2146e+04  4e+00  5e-04  6e-07
12: -1.2144e+04 -1.2145e+04  2e+00  1e-04  2e-07
13: -1.2145e+04 -1.2145e+04  5e-01  2e-05  3e-08
14:

In [ ]:
from readme_parser import rparser
rparser()

Parsing Readme files


In [ ]:
# Train a tree-based model on the merged WESAD feature table and rank features by importance
!python tree_feature_importance.py \
--data data/m14_merged.csv \
--output-dir artifacts \
--n-splits 5 --n-estimators 500

Saved outputs to /content/artifacts


In [ ]:
# !python tree_feature.py \
#   --data data/m14_merged.csv \
#   --output-dir artifacts2 \
#   --n-splits 5 \
#   --n-estimators 500 \
#   --drop-cols

In [ ]:
# Train RandomForest and XGBoost after removing the requested features
# This loads the wrangled CSV, drops the columns, runs 5-fold Stratified CV, and prints accuracy and F1-macro.

#%cd /content/WESAD

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# 1) Load data
df = pd.read_csv("data/m14_merged.csv")

# 2) Columns to drop
to_remove = ['EDA_tonic_min', 'ACC_x_min', 'ACC_y_min', 'ACC_z_min', 'TEMP_min', 'BVP_min']
label_col = 'label'  # change if your label column is different

# 3) Prepare X, y
drop_cols = [c for c in to_remove if c in df.columns]
X = df.drop(columns=[label_col] + drop_cols, errors='ignore')
y = df[label_col].astype(int)

print("Dropped columns:", drop_cols)
print("X shape:", X.shape)

# 4) 5-fold Stratified CV
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rf_accs, rf_f1s = [], []
xgb_accs, xgb_f1s = [], []

for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
    X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    # RandomForest
    rf = RandomForestClassifier(
        n_estimators=500,
        random_state=42,
        n_jobs=-1
    )
    rf.fit(X_tr, y_tr)
    rf_pred = rf.predict(X_va)
    rf_accs.append(accuracy_score(y_va, rf_pred))
    rf_f1s.append(f1_score(y_va, rf_pred, average='macro'))

    # XGBoost
    xgb = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        tree_method='hist',
        eval_metric='logloss'
    )
    xgb.fit(X_tr, y_tr)
    xgb_pred = xgb.predict(X_va)
    xgb_accs.append(accuracy_score(y_va, xgb_pred))
    xgb_f1s.append(f1_score(y_va, xgb_pred, average='macro'))

print(f"RandomForest — Accuracy: {np.mean(rf_accs):.4f} ± {np.std(rf_accs):.4f} | F1-macro: {np.mean(rf_f1s):.4f} ± {np.std(rf_f1s):.4f}")
print(f"XGBoost     — Accuracy: {np.mean(xgb_accs):.4f} ± {np.std(xgb_accs):.4f} | F1-macro: {np.mean(xgb_f1s):.4f} ± {np.std(xgb_f1s):.4f}")


# Save the reduced-feature results to artifacts
import json, os
os.makedirs("artifacts", exist_ok=True)
results = {
    "dropped_columns": drop_cols,
    "rf_accuracy_mean": float(np.mean(rf_accs)),
    "rf_accuracy_std": float(np.std(rf_accs)),
    "rf_f1_macro_mean": float(np.mean(rf_f1s)),
    "rf_f1_macro_std": float(np.std(rf_f1s)),
    "xgb_accuracy_mean": float(np.mean(xgb_accs)),
    "xgb_accuracy_std": float(np.std(xgb_accs)),
    "xgb_f1_macro_mean": float(np.mean(xgb_f1s)),
    "xgb_f1_macro_std": float(np.std(xgb_f1s)),
}
with open("artifacts/reduced_features_results.json", "w") as f:
    json.dump(results, f, indent=2)
print("Saved to artifacts/reduced_features_results.json")

Dropped columns: ['EDA_tonic_min', 'ACC_x_min', 'ACC_y_min', 'ACC_z_min', 'TEMP_min', 'BVP_min']
X shape: (1178, 52)
RandomForest — Accuracy: 0.9762 ± 0.0091 | F1-macro: 0.9734 ± 0.0122
XGBoost     — Accuracy: 0.9839 ± 0.0073 | F1-macro: 0.9837 ± 0.0093
Saved to artifacts/reduced_features_results.json
